In [1]:
# Imports & setup

import os, re, numpy as np, pandas as pd, tensorflow as tf
from nltk import download as nltk_download
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

'''
Sets the seed for TensorFlow’s internal random number generator
TensorFlow uses randomness in many places: Initializing weights, Shuffling data, Dropout layers;
Setting this makes sure that every time running the notebook, we'll get the same model initialization and training behavior
'''
tf.random.set_seed(42) # common placeholder seed value
np.random.seed(42) # helps ensure NumPy-based random behavior is the same across runs.

nltk_download("punkt")
nltk_download("stopwords")


[nltk_data] Downloading package punkt to /Users/yifanchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yifanchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
'''Pre-processing'''

# Create a customised stop‑word set that *keeps* negation words
STOP_WORDS = set(stopwords.words("english"))
# Keep negation words so sentiment isn’t flipped
NEGATION_WORDS = {"no", "not", "nor", "cannot", "can't", "won't", "n't"}
CUSTOM_STOPWORDS = STOP_WORDS - NEGATION_WORDS

def preprocess_text(text: str) -> str:
    '''
    Clean a raw tweet and return a whitespace-separated token string.

    Parameters
    ----------
    text : str
        Raw tweet text (may include URLs, mentions, hashtags, punctuation, etc.).

    Returns
    -------
    str
        Cleaned, lower-cased, stop-word-filtered text where:
        * URLs and web addresses are removed
        * Mentions (@username) and hashtags (#topic) are stripped
        * Non-alphabetic characters are replaced by spaces
        * Standard NLTK stop-words **except negations** are removed
        * Tokens are reunited by single spaces
    '''
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"@\w+|#", "", text) # strip mentions & hashtags
    text = re.sub(r"[^a-zA-Z]", " ", text) # keep letters only
    tokens = word_tokenize(text.strip()) # tokenize
    tokens = [tok for tok in tokens if tok not in CUSTOM_STOPWORDS] # drop stop‑words (negations kept)
    return " ".join(tokens)


In [3]:
'''Load & clean'''

CSV_PATH = "movie.csv" # dataset file to be tested

# Read the raw dataset into a DataFrame
df = pd.read_csv(CSV_PATH)
# Apply the text cleaner to every tweet
df["cleaned_text"] = df["text"].apply(preprocess_text)

# Split out features and labels
texts  = df["cleaned_text"].values
labels = df["label"].values 
print("Dataset size:", len(df))


Dataset size: 40000


In [4]:
'''Train/Test split + label encoding'''

# Split so every model sees the exact same data partitions
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.20, random_state=42, stratify=labels
)

# Map string labels to integer IDs
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc  = label_encoder.transform(y_test)

num_classes  = len(label_encoder.classes_)
y_train_cat  = to_categorical(y_train_enc, num_classes)
y_test_cat   = to_categorical(y_test_enc,  num_classes)

print("Classes →", label_encoder.classes_)


Classes → [0 1]


In [5]:
'''Tokeniser & padding'''

# Hyper‑parameters for the tokeniser and sequences
MAX_VOCAB, MAX_SEQ_LEN, EMBED_DIM = 10_000, 100, 64 # max_vocab limits the size of the vocabulary that the model is allowed to learn

# Build and fit the tokeniser on train texts only 
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert texts to integer sequences and pad sequences to uniform length
X_train_pad = pad_sequences(tokenizer.texts_to_sequences(X_train),
                            maxlen=MAX_SEQ_LEN, padding="post")
X_test_pad  = pad_sequences(tokenizer.texts_to_sequences(X_test),
                            maxlen=MAX_SEQ_LEN, padding="post")

print("Vocab size (training):", len(tokenizer.word_index))


Vocab size (training): 82552


In [6]:
'''Build Bidirectional-LSTM architecture '''

# Define the model sequentially; add input_shape so Keras builds immediately
model = Sequential([
    Embedding(input_dim=MAX_VOCAB, # vocabulary cap
              output_dim=EMBED_DIM, # embedding vector length
              input_shape=(MAX_SEQ_LEN,)),  # one integer per time‑step
    Bidirectional(LSTM(64)),  # 64 units forward + 64 backward
    Dropout(0.5), # regularisation
    Dense(num_classes, activation="softmax") # 3‑way sentiment output
])

# Compile with categorical cross‑entropy & Adam optimiser
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary() 


/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env_py311/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 706,306 (2.69 MB)

 Trainable params: 706,306 (2.69 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
'''Compute balanced weights for each class'''

# Balanced weighting: each class contributes equally to loss
weights = compute_class_weight("balanced",
                               classes=np.unique(y_train_enc),
                               y=y_train_enc)
class_weights = dict(enumerate(weights))
print("Class weights:", class_weights)


Class weights: {0: 0.9990633780830471, 1: 1.0009383797309979}


In [8]:
# Train with early stopping

early_stop = EarlyStopping(
	monitor="val_loss",  # watch validation loss
	patience=3, # stop after 3 epochs w/o improvement
    restore_best_weights=True) # roll back to the best epoch

history = model.fit(
    X_train_pad, y_train_cat,
    epochs=15, batch_size=32,
    validation_split=0.2, # hold out 20 % of train as val
    class_weight=class_weights, # handle class imbalance
    callbacks=[early_stop],
    verbose=2 # 1 line/epoch
)


Epoch 1/15
800/800 - 28s - 34ms/step - accuracy: 0.8026 - loss: 0.4158 - val_accuracy: 0.8703 - val_loss: 0.3231
Epoch 2/15
800/800 - 28s - 35ms/step - accuracy: 0.9090 - loss: 0.2362 - val_accuracy: 0.8652 - val_loss: 0.3603
Epoch 3/15
800/800 - 28s - 35ms/step - accuracy: 0.9446 - loss: 0.1565 - val_accuracy: 0.8622 - val_loss: 0.4322
Epoch 4/15
800/800 - 28s - 35ms/step - accuracy: 0.9575 - loss: 0.1212 - val_accuracy: 0.8605 - val_loss: 0.4559


In [11]:
'''Evaluation & classification report'''

# Evaluate on the reserved test partition
test_loss, test_acc = model.evaluate(X_test_pad, y_test_cat, verbose=0)
print(f"Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")

y_pred = model.predict(X_test_pad, verbose=0).argmax(axis=1)

print("\nClassification report:\n")
print(classification_report(
    y_test_enc,
    y_pred,
    target_names=["negative","positive"]     # ← instead of label_encoder.classes_
))
print("Macro-F1:", f1_score(y_test_enc, y_pred, average="macro"))


Test accuracy: 0.8673 | Test loss: 0.3295

Classification report:

              precision    recall  f1-score   support

    negative       0.84      0.91      0.87      4004
    positive       0.90      0.82      0.86      3996

    accuracy                           0.87      8000
   macro avg       0.87      0.87      0.87      8000
weighted avg       0.87      0.87      0.87      8000

Macro-F1: 0.8669776283802102


In [12]:
# '''Save predictions to CSV'''

# pred_df = pd.DataFrame({
#     "text"           : X_test,
#     "airline_sentiment"     : label_encoder.inverse_transform(y_test_enc),
#     "predicted_sentiment": label_encoder.inverse_transform(y_pred)
# })
# pred_df.to_csv("tf_predictions_movies.csv", index=False)
# print("Predictions saved → tf_predictions_movies.csv")


Predictions saved → tf_predictions_movies.csv
